<a href="https://colab.research.google.com/github/laiamr/tfm/blob/main/Scripts/TFM_Pipeline_2_WordSenseDisambiguation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn groq langchain langchain-core langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from collections import Counter
from google.colab import userdata
from groq import Groq
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
# Datafile urls
annotation_data_url = 'https://raw.githubusercontent.com/laiamr/tfm_temp/refs/heads/main/Data/annotated_sample.csv'
complete_data_url = 'https://raw.githubusercontent.com/laiamr/tfm_temp/refs/heads/main/Data/cleanDataset.csv'

**LOAD THE DATA**

In [ ]:
# Get data from files
df_ann = pd.read_csv(annotation_data_url, sep=";")

In [ ]:
print(df_ann.shape)
df_ann.head()

(300, 7)


,id,searchQuery,text,timestamp,cleanText,foundWord,socialMediaSense
0,1620047577906298880,m'agrada,no m'agrada de fer servir el mot 'correcte'. d...,2023-01-30 13:13:00+00:00,no m'agrada de fer servir el mot 'correcte'. d...,m'agrada,0
1,1620099264805486592,respondre,si la majoria de catalunya decideix que vol ro...,2023-01-30 16:38:00+00:00,si la majoria de catalunya decideix que vol ro...,respondre,0
2,1620116848825217024,retuit,et segueixo i no puc fer retuit a les teves pi...,2023-01-30 17:48:00+00:00,et segueixo i no puc fer retuit a les teves pi...,retuit,1
3,1652264071687806977,tuits,sí.\nsolen ser tuits patrocinats. \nde pagament.,2023-04-29T10:50:00.000Z,sí. solen ser tuits patrocinats. de pagament.,tuits,1
4,1652412536573108225,amic,gràcies amic!! ✊🏻,2023-04-29T20:40:00.000Z,gràcies amic!! ✊🏻,amic,0


In [ ]:
Counter(df_ann['socialMediaSense'])

Counter({0: 195, 1: 105})

In [ ]:
# Develop (test prompting) - test dataset split (200 - 100)
df_train, df_test = train_test_split(df_ann, train_size=200, random_state=42, stratify=df_ann['socialMediaSense'])
print(df_train.shape)
print(df_test.shape)

(200, 7)
(100, 7)


**SET UP THE MODEL**

In [ ]:
# Get API key and set the model name
try:
  api_key=userdata.get("GROQ_API_KEY")
except:
  api_key=input("Please enter your GROQ API key: ")

model_name_1="meta-llama/llama-4-scout-17b-16e-instruct"
model_name_2="qwen-qwq-32b"
model_name_3="llama-3.3-70b-versatile"
model_name_4="gemma2-9b-it"
model_name_5="qwen/qwen3-32b"

In [ ]:
# Initialize models with Groq
model_1=ChatGroq(groq_api_key=api_key,
                 model_name=model_name_1)

model_2=ChatGroq(groq_api_key=api_key,
                 model_name=model_name_2,
                 temperature=0.7,
                 model_kwargs={'reasoning_format':'hidden'})

model_3=ChatGroq(groq_api_key=api_key,
                 model_name=model_name_3)

model_4=ChatGroq(groq_api_key=api_key,
                 temperature=0.7,
                 model_name=model_name_4)

model_5=ChatGroq(groq_api_key=api_key,
                 model_name=model_name_5,
                 temperature=0.7,
                 model_kwargs={'reasoning_format':'hidden'})

**First try with prompts - not the best**

In [ ]:
# Function to query the first model with our prompt
def query_model_1(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil."),
      HumanMessage(content= f"""Digues si la paraula "{keyword}" del text següent està parlant sobre xarxes socials. Respon només amb el valor '1' en cas afirmatiu, i '0' en cas negatiu.
                      Text: {text}.
                      Exemples:
                      'retuitar' al text 'perquè, retuitar? perque fa falta' -> 1
                      'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> 0
                      Resposta ->
                      \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_1.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the second model with our prompt
def query_model_2(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil."),
      HumanMessage(content= f"""Digues si la paraula "{keyword}" del text següent està parlant sobre xarxes socials. Respon només amb el valor '1' en cas afirmatiu, i '0' en cas negatiu.
                      Text: {text}.
                      Exemples:
                      'retuitar' al text 'perquè, retuitar? perque fa falta' -> 1
                      'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> 0
                      Resposta ->
                      \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_2.invoke(prompt)

  # Extract the response
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the second model with our prompt
def query_model_3_noOtherLabels(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil."),
      HumanMessage(content= f"""Digues si la paraula "{keyword}" del text següent està parlant sobre xarxes socials. Respon només amb el valor '1' en cas afirmatiu, i '0' en cas negatiu.
                      Text: {text}.
                      Exemples:
                      'retuitar' al text 'perquè, retuitar? perque fa falta' -> 1
                      'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> 0
                      Resposta ->
                      \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_3.invoke(prompt)

  # Extract the response
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the third model with our prompt and the previous data
def query_model_3(keyword: str, text: str, label_model_1: int, label_model_2: int) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media
  taking into account the labels produced by two different models.
  Create the prompt to disambiguate the sense using the keyword and tweet using the labels from two other models

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears
    label_model_1 (int): the label produced by one of the models
    label_model_2 (int): the label produced by the other model

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil."),
      HumanMessage(content= f"""Has de decidir si la paraula "{keyword}" del text següent està parlant sobre xarxes socials. Respon només amb el valor '1' en cas afirmatiu, i '0' en cas negatiu.
                      Text: {text}.
                      Dos altres models han fet les valoracions següents. Model 1: {label_model_1}, Model 2: {label_model_2}.
                      Exemples:
                      'retuitar' al text 'perquè, retuitar? perque fa falta' -> 1
                      'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> 0
                      Resposta ->
                      \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_3.invoke(prompt)

  # Extract the response
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

**First prompt with the 3 models (all independent labels + one 'agentic')**

In [ ]:
# EXECUTE PROMPTING WITH FIRST PROMPTS
# Initialise empty list to save final labels
mod1_labels = []
mod2_labels = []
mod3_labels = []
llm_labels = []
# Loop through each row of the labeled dataset to get the model's annotation
for _, row in df_train.iterrows():
  # Ask the first model for the label
  lab_1 = query_model_1(row['foundWord'], row['cleanText'])
  mod1_labels.append(lab_1)
  #print(lab_1)
  # Ask the second model for the label
  lab_2 = query_model_2(row['foundWord'], row['cleanText'])
  mod2_labels.append(lab_2)
  #print(lab_2)
  # Ask the third model for the label, without the other ones
  lab_3 = query_model_3_noOtherLabels(row['foundWord'], row['cleanText'])
  mod3_labels.append(lab_3)
  #print(lab_3)
  # Ask the third model for the final decision on the label
  lab_fin = query_model_3(row['foundWord'], row['cleanText'], lab_1, lab_2)
  #print(lab_fin)
  llm_labels.append(lab_fin)
  #print("-----")


**PERFORMANCE OF FIRST PROMPT**

In [ ]:
# MODEL 1
print(accuracy_score(df_train['socialMediaSense'], mod1_labels))
print(f1_score(df_train['socialMediaSense'], mod1_labels, average=))
print(classification_report(df_train['socialMediaSense'], mod1_labels))

0.845
0.7919463087248322
              precision    recall  f1-score   support

           0       0.91      0.85      0.88       130
           1       0.75      0.84      0.79        70

    accuracy                           0.84       200
   macro avg       0.83      0.84      0.83       200
weighted avg       0.85      0.84      0.85       200



In [ ]:
# MODEL 2
print(accuracy_score(df_train['socialMediaSense'], mod2_labels))
print(f1_score(df_train['socialMediaSense'], mod2_labels))
print(classification_report(df_train['socialMediaSense'], mod2_labels))

0.895
0.8531468531468531
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       130
           1       0.84      0.87      0.85        70

    accuracy                           0.90       200
   macro avg       0.88      0.89      0.89       200
weighted avg       0.90      0.90      0.90       200



In [ ]:
# MODEL 3 (no other labels)
print(accuracy_score(df_train['socialMediaSense'], mod3_labels))
print(f1_score(df_train['socialMediaSense'], mod3_labels))
print(classification_report(df_train['socialMediaSense'], mod3_labels))

0.865
0.7969924812030075
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       130
           1       0.84      0.76      0.80        70

    accuracy                           0.86       200
   macro avg       0.86      0.84      0.85       200
weighted avg       0.86      0.86      0.86       200



In [ ]:
# MODEL 3 (final decision)
print(accuracy_score(df_train['socialMediaSense'], llm_labels))
print(f1_score(df_train['socialMediaSense'], llm_labels))
print(classification_report(df_train['socialMediaSense'], llm_labels))

0.865
0.7969924812030075
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       130
           1       0.84      0.76      0.80        70

    accuracy                           0.86       200
   macro avg       0.86      0.84      0.85       200
weighted avg       0.86      0.86      0.86       200



**SECOND TRY AT PROMPTING - MORE EXAMPLES AND REFORMULATED THE TEXT - THE BEST ONES**

In [ ]:
# Function to query the first model with our prompt
def query_model_1_V2(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_1.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the second model with our prompt
def query_model_2_V2(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_2.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# This model gets the same input as the others, but also the previous labels, so it can use those in the final decision
def query_model_3_V2(keyword: str, text: str, label_model_1: int, label_model_2: int) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media
  taking into account the labels produced by two different models.
  Create the prompt to disambiguate the sense using the keyword and tweet using the labels from two other models

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears
    label_model_1 (int): the label produced by one of the models
    label_model_2 (int): the label produced by the other model

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      A més, també tens disponibles les respostes que han donat dos models diferents. Tingues en compte aquests valors quan prenguis la decisió.
      Resposta model 1: {label_model_1}
      Resposta model 2: {label_model_2}

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_3.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the third model with our prompt (no final decision, just another model labeling)
def query_model_3_V2_noOtherLabels(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_3.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the forth model with our prompt
def query_model_4_V2(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_4.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

In [ ]:
# Function to query the fifth model with our prompt
def query_model_5_V2(keyword: str, text: str) -> int:
  """
  We ask the model to determine if a word in a text is used in the context of social media.
  Create the prompt to disambiguate the sense using the keyword and tweet.

  Parameters:
    keyword (str): the word we want to disambiguate
    text (str): the context in which the keyword appears

  Return:
    int: 1 for yes (the context is social media related) or 0 otherwise.
  """
  # Create query
  prompt = [
      SystemMessage(content= "Ets un assistent lingüístic útil. Les úniques respostes possibles son els valors 0 o 1."),
      HumanMessage(content= f"""Et donaré una paraula i un text i has de decidir si aquesta paraula té un significat
      relacionat amb les xarxes socials. És a dir, si la paraula es fa servir dins el text per parlar d'algun element de
      les xarxes socials o d'alguna interacció amb les xarxes socials o altres usuaris de la xarxa social.

      Et poso alguns exemples a continuació.

      Exemples:

      paraula 'retuitar' al text 'perquè, retuitar? perque fa falta' -> Resposta = 1

      paraula 'like' al text 'Acabo de fer un like al teu tuit! Ja en tens 15!' -> Resposta = 1

      paraula 'bloquejar' al text 'el mosén i el seu ego van bloquejar qualsevol acord.' -> Resposta = 0

      paraula 'comentaris' al text 'La Lucy podrà tornar a ser lliure? Què en penses tu? Creus que podrà tornar a volar ben lluny? Us llegim als comentaris!' -> Resposta = 1

      paraula 'seguir' al text 'Ja tenim la línia de tempestes present al territori. La línia es dessplaça cap a l'est i es pot seguir clarament a les imatges de radar' -> Resposta = 0

      paraula 'publicació' al text 'Tot i que  encara haurem d'esperar a la publicació al DOGC per veur'n la lletra petita, estem molt satisfets de que per fi s'hagi modificat el Decret 16/2019' -> Resposta = 0

      paraula 'missatge' al text 'El missatge que veurem als lavabo del RCDE Stadium si torna @GerardMoreno9.' -> Resposta = 0

      paraula 'follow' al text 'No et perdis re, fes follow' -> Resposta = 1

      paraula 'història' al text 'No ho oblidarem mai. I no permetrem que la història ho oblidi' -> Resposta = 0

      paraula 'piulada' al text 'Vull conèixer la persona que ha programat X per a que surti l'anunci de Temv a cada piulada...' -> Resposta = 1

      Recorda, has de dir si la paraula del text següent parla de xarxes socials. Respon només amb el valor 1 en cas afirmatiu i
      respon amb el valor 0 en cas negatiu.

      Paraula: {keyword}
      Text: {text}

      Resposta = \n""")
  ]
  #print(prompt)

  # Query the model with the prompt
  response = model_5.invoke(prompt)

  # Extract the response's content and remove whitespaces
  value = response.content.strip()
  #print(value)

  # If the model returns more than just the label (1 or 0), try to extract it from the text
  options = re.findall(r"\d", value, flags=re.IGNORECASE) # look for all digits in the response
  if(len(options) > 1):
    label = int(options[-1]) # if there is more than one digit in the LLM response, we'll take the last one.
  elif(len(options) == 1):
    label = int(options[0]) # if there is only one, we take it
  else:
    label = 0 # if there was no match, we say no

  return label

**SECOND prompt with the 5 models (all independent labels + one case of 'agentic' approach (llama Scout + qwen QwQ + llama 70b)**

In [ ]:
# EXECUTE PROMPTING
# Initialise empty list to save final labels
mod1_labels_V2 = []
mod2_labels_V2 = []
llm_labels_V2 = []
mod3_labels_V2 = []
mod4_labels_V2 = []
mod5_labels_V2 = []
# Loop through each row of the labeled dataset to get the model's annotation
for _, row in df_train.iterrows():

  # Ask the first model for the label
  lab_1 = query_model_1_V2(row['foundWord'], row['cleanText'])
  mod1_labels_V2.append(lab_1)

  # Ask the second model for the label
  lab_2 = query_model_2_V2(row['foundWord'], row['cleanText'])
  mod2_labels_V2.append(lab_2)

  # Ask the third model for the final decision on the label
  lab_fin = query_model_3_V2(row['foundWord'], row['cleanText'], lab_1, lab_2)
  llm_labels_V2.append(lab_fin)

  # Ask the third model for the label (no other labels)
  lab_3_noLabels = query_model_3_V2_noOtherLabels(row['foundWord'], row['cleanText'])
  mod3_labels_V2.append(lab_3_noLabels)

  # Ask the forth model for the label (no other labels)
  lab_4 = query_model_4_V2(row['foundWord'], row['cleanText'])
  mod4_labels_V2.append(lab_4)

  # Ask the fifth model for the label (no other labels)
  lab_5 = query_model_5_V2(row['foundWord'], row['cleanText'])
  mod5_labels_V2.append(lab_5)

**PERFORMANCE OF SECOND PROMPT**

WITH 'TRAIN' SET

In [ ]:
# MODEL 1
print(accuracy_score(df_train['socialMediaSense'], mod1_labels_V2))
print(f1_score(df_train['socialMediaSense'], mod1_labels_V2))
print(classification_report(df_train['socialMediaSense'], mod1_labels_V2))

0.83
0.7951807228915663
              precision    recall  f1-score   support

           0       0.96      0.77      0.85       130
           1       0.69      0.94      0.80        70

    accuracy                           0.83       200
   macro avg       0.82      0.86      0.82       200
weighted avg       0.87      0.83      0.83       200



In [ ]:
# MODEL 2 - QWEN 32b
print(accuracy_score(df_train['socialMediaSense'], mod2_labels_V2))
print(f1_score(df_train['socialMediaSense'], mod2_labels_V2))
print(classification_report(df_train['socialMediaSense'], mod2_labels_V2))

0.92
0.8873239436619719
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       130
           1       0.88      0.90      0.89        70

    accuracy                           0.92       200
   macro avg       0.91      0.92      0.91       200
weighted avg       0.92      0.92      0.92       200



In [ ]:
# MODEL 3
print(accuracy_score(df_train['socialMediaSense'], llm_labels_V2))
print(f1_score(df_train['socialMediaSense'], llm_labels_V2))
print(classification_report(df_train['socialMediaSense'], llm_labels_V2))

0.875
0.8407643312101911
              precision    recall  f1-score   support

           0       0.96      0.84      0.90       130
           1       0.76      0.94      0.84        70

    accuracy                           0.88       200
   macro avg       0.86      0.89      0.87       200
weighted avg       0.89      0.88      0.88       200



In [ ]:
# MODEL 3 (no other labels) - LLAMA3.3 70b
print(accuracy_score(df_train['socialMediaSense'], mod3_labels_V2))
print(f1_score(df_train['socialMediaSense'], mod3_labels_V2))
print(classification_report(df_train['socialMediaSense'], mod3_labels_V2))

0.875
0.8427672955974843
              precision    recall  f1-score   support

           0       0.97      0.83      0.90       130
           1       0.75      0.96      0.84        70

    accuracy                           0.88       200
   macro avg       0.86      0.89      0.87       200
weighted avg       0.90      0.88      0.88       200



In [ ]:
# MODEL 4
print(accuracy_score(df_train['socialMediaSense'], mod4_labels_V2))
print(f1_score(df_train['socialMediaSense'], mod4_labels_V2))
print(classification_report(df_train['socialMediaSense'], mod4_labels_V2))

0.73
0.7032967032967034
              precision    recall  f1-score   support

           0       0.93      0.63      0.75       130
           1       0.57      0.91      0.70        70

    accuracy                           0.73       200
   macro avg       0.75      0.77      0.73       200
weighted avg       0.81      0.73      0.74       200



In [ ]:
# MODEL 5
print(accuracy_score(df_train['socialMediaSense'], mod5_labels_V2))
print(f1_score(df_train['socialMediaSense'], mod5_labels_V2))
print(classification_report(df_train['socialMediaSense'], mod5_labels_V2))

0.88
0.8285714285714286
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       130
           1       0.83      0.83      0.83        70

    accuracy                           0.88       200
   macro avg       0.87      0.87      0.87       200
weighted avg       0.88      0.88      0.88       200



In [ ]:
# MODEL 2, Qwen QwQ 32b model is the one with highest accuracy and F1

WITH 'TEST' SET - we try with the top 3 performers in the training evaluation

In [ ]:
# NOW WITH THE TEST SET
# Initialise empty list to save final labels
mod2_labels_V2_test = []
mod3_labels_V2_test = []
mod5_labels_V2_test = []
# Loop through each row of the labeled dataset to get the model's annotation
for _, row in df_test.iterrows():

  # Ask the second model for the label
  lab_2_test = query_model_2_V2(row['foundWord'], row['cleanText'])
  mod2_labels_V2_test.append(lab_2_test)

  # Ask the third model for the label (no other labels)
  lab_3_noLabels_test = query_model_3_V2_noOtherLabels(row['foundWord'], row['cleanText'])
  mod3_labels_V2_test.append(lab_3_noLabels_test)

  # Ask the fifth model for the label (no other labels)
  lab_5_test = query_model_5_V2(row['foundWord'], row['cleanText'])
  mod5_labels_V2_test.append(lab_5_test)

In [ ]:
# MODEL 2 - TEST
print(accuracy_score(df_test['socialMediaSense'], mod2_labels_V2_test))
print(f1_score(df_test['socialMediaSense'], mod2_labels_V2_test))
print(classification_report(df_test['socialMediaSense'], mod2_labels_V2_test))

0.92
0.8947368421052632
              precision    recall  f1-score   support

           0       0.98      0.89      0.94        65
           1       0.83      0.97      0.89        35

    accuracy                           0.92       100
   macro avg       0.91      0.93      0.92       100
weighted avg       0.93      0.92      0.92       100



In [ ]:
# MODEL 3 - TEST
print(accuracy_score(df_test['socialMediaSense'], mod3_labels_V2_test))
print(f1_score(df_test['socialMediaSense'], mod3_labels_V2_test))
print(classification_report(df_test['socialMediaSense'], mod3_labels_V2_test))

0.89
0.8571428571428571
              precision    recall  f1-score   support

           0       0.97      0.86      0.91        65
           1       0.79      0.94      0.86        35

    accuracy                           0.89       100
   macro avg       0.88      0.90      0.88       100
weighted avg       0.90      0.89      0.89       100



In [ ]:
# MODEL 5 - TEST
print(accuracy_score(df_test['socialMediaSense'], mod5_labels_V2_test))
print(f1_score(df_test['socialMediaSense'], mod5_labels_V2_test))
print(classification_report(df_test['socialMediaSense'], mod5_labels_V2_test))

0.9
0.8611111111111112
              precision    recall  f1-score   support

           0       0.94      0.91      0.92        65
           1       0.84      0.89      0.86        35

    accuracy                           0.90       100
   macro avg       0.89      0.90      0.89       100
weighted avg       0.90      0.90      0.90       100



**THE BEST MODEL IN BOTH TRAINING AND TESTING DATASETS IS QWEN QWQ 32B (MODEL 2)**

In [ ]:
# Get the whole dataset
df = pd.read_csv(complete_data_url, sep=";")
print(df.shape)
df.head()

(7041, 6)


,id,searchQuery,text,timestamp,cleanText,foundWord
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,"un ruzi*, que he barrat tot d'una, vota pel me...",repiulet
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,podeu demanar la dimissió de sigfrid gras sens...,retuitar
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,"perquè, retuitar? perque fa falta",retuitar
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,"en contra de retuitar genocides, per molt sucó...",retuitar
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,retuit si tu també creus que tampoc s'ha de re...,retuit


In [ ]:
# Merge df and df_ann by the columns id and foundWord
df_merged = pd.merge(df, df_ann[['id', 'foundWord', 'socialMediaSense']], on=['id', 'foundWord'], how='left')
print(df_merged.shape)
df_merged.head()

(7041, 7)


,id,searchQuery,text,timestamp,cleanText,foundWord,socialMediaSense
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,"un ruzi*, que he barrat tot d'una, vota pel me...",repiulet,NaN
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,podeu demanar la dimissió de sigfrid gras sens...,retuitar,NaN
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,"perquè, retuitar? perque fa falta",retuitar,NaN
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,"en contra de retuitar genocides, per molt sucó...",retuitar,NaN
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,retuit si tu també creus que tampoc s'ha de re...,retuit,NaN


In [ ]:
# Get the NA values from the socialMediaSense column to annotate them
df_unlab = df_merged[df_merged['socialMediaSense'].isna()]

In [ ]:
df_unlab.shape

(6741, 7)

In [ ]:
# Initialise empty list to save final labels for the remaining unlabeled data
mod2_labels_V2_full = []
# Loop through each row of the unlabeled dataset to get the model's annotation
for _, row in df_unlab.iterrows():
  # Ask the fifth model for the label (no other labels)
  lab_2_full = query_model_2_V2(row['foundWord'], row['cleanText'])
  mod2_labels_V2_full.append(lab_2_full)

In [ ]:
len(mod2_labels_V2_full)

6741

In [ ]:
df_unlab['socialMediaSense'] = mod2_labels_V2_full

<ipython-input-26-674219691>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlab['socialMediaSense'] = mod2_labels_V2_full


In [ ]:
df_unlab.head()

,id,searchQuery,text,timestamp,cleanText,foundWord,socialMediaSense
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,"un ruzi*, que he barrat tot d'una, vota pel me...",repiulet,1
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,podeu demanar la dimissió de sigfrid gras sens...,retuitar,1
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,"perquè, retuitar? perque fa falta",retuitar,1
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,"en contra de retuitar genocides, per molt sucó...",retuitar,1
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,retuit si tu també creus que tampoc s'ha de re...,retuit,1


In [ ]:
# Concat the two dataframes with the labels
# The manually annotated and the automated one
df_final = pd.concat([df_ann, df_unlab])
print(df_final.shape)
df_final.head()

(7041, 7)


,id,searchQuery,text,timestamp,cleanText,foundWord,socialMediaSense
0,1620047577906298880,m'agrada,no m'agrada de fer servir el mot 'correcte'. d...,2023-01-30 13:13:00+00:00,no m'agrada de fer servir el mot 'correcte'. d...,m'agrada,0
1,1620099264805486592,respondre,si la majoria de catalunya decideix que vol ro...,2023-01-30 16:38:00+00:00,si la majoria de catalunya decideix que vol ro...,respondre,0
2,1620116848825217024,retuit,et segueixo i no puc fer retuit a les teves pi...,2023-01-30 17:48:00+00:00,et segueixo i no puc fer retuit a les teves pi...,retuit,1
3,1652264071687806977,tuits,sí.\nsolen ser tuits patrocinats. \nde pagament.,2023-04-29T10:50:00.000Z,sí. solen ser tuits patrocinats. de pagament.,tuits,1
4,1652412536573108225,amic,gràcies amic!! ✊🏻,2023-04-29T20:40:00.000Z,gràcies amic!! ✊🏻,amic,0


In [ ]:
# Save final dataframe to csv
df_final.to_csv("Annotated_data.csv", sep=";", index=False)

In [ ]:
Counter(df_final['socialMediaSense'])

Counter({0: 4132, 1: 2909})

In [ ]:
# Filter the data to keep only the ones that are related to social media (socialMediaSense = 1)
df_socialMediaSense = df_final[df_final['socialMediaSense'] == 1]
print(df_socialMediaSense.shape)
df_socialMediaSense.head()

(2909, 7)


,id,searchQuery,text,timestamp,cleanText,foundWord,socialMediaSense
2,1620116848825217024,retuit,et segueixo i no puc fer retuit a les teves pi...,2023-01-30 17:48:00+00:00,et segueixo i no puc fer retuit a les teves pi...,retuit,1
3,1652264071687806977,tuits,sí.\nsolen ser tuits patrocinats. \nde pagament.,2023-04-29T10:50:00.000Z,sí. solen ser tuits patrocinats. de pagament.,tuits,1
5,1619801223661748226,comentat,amb quin mobil has comentat?,2023-01-29T20:54:00.000Z,amb quin mobil has comentat?,comentat,1
9,1718708884951409072,blocat,fent mongetes i de cap al compte b q estic (or...,2023-10-29T19:18:00.000Z,fent mongetes i de cap al compte b q estic (or...,blocat,1
10,1620159160997355520,tuit,no passa res... un tuit i a dormir.\nfeina fet...,2023-01-30 20:36:00+00:00,no passa res... un tuit i a dormir. feina feta...,tuit,1


In [ ]:
# Save the social media data into a csv to continue the analysis
df_socialMediaSense.to_csv("SocialMedia_Data.csv", sep=";", index=False)